https://huggingface.co/agents-course/notebooks/blob/main/unit2/llama-index/agents.ipynb

https://docs.llamaindex.ai/en/stable/getting_started/async_python/

https://docs.llamaindex.ai/en/stable/examples/agent/agent_workflow_basic/

https://docs.llamaindex.ai/en/stable/understanding/agent/

In [ ]:
# !pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api -U -q

In [ ]:
from huggingface_hub import login

login()

In [ ]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.core.agent.workflow import (
    AgentStream,
    AgentWorkflow,
    ReActAgent,
    ToolCallResult,
)
from llama_index.core.tools import QueryEngineTool
from llama_index.core.workflow import Context
from llama_index.embeddings.huggingface import HuggingFaceInferenceAPIEmbeddings
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.vector_stores.chroma import ChromaVectorStore


# Initialising Agents

In [57]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b


# llm = MLXLLM(model_name="mlx-community/Qwen2.5-Coder-3B-Instruct-bf16")
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-3B-Instruct")


agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[subtract, multiply, divide, add],
    llm=llm,
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools.",
)


In [58]:
handler = agent.run("What is (2 + 2) * 2?")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 2, "b": 2}
Observation: 4
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {"a": 4, "b": 2}
Observation: 8
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 8
Called tool:  add {'a': 2, 'b': 2} => 4
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 4, 'b': 2}
Called tool:  multiply {'a': 4, 'b': 2} => 8
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 8

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='8')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 2, 'b': 2}, tool_id='df0977b0-e294-4496-9052-66b181b0902b', tool_output=ToolOutput(content='4', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False), ToolCallResult(tool_name='multiply', tool_kwargs={'a': 4, 'b': 2}, tool_id='6e3a94fc-47b2-4230-928b-f5aeb8b14ac3', tool_output=ToolOutput(content='8', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 4, 'b': 2}}, raw_output=8, is_error=False), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='8', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1742761495, id='', model='Qwen/Qwen2.5-Coder-3B-Instruct', system_fingerprint='

In [59]:
resp

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='8')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 2, 'b': 2}, tool_id='df0977b0-e294-4496-9052-66b181b0902b', tool_output=ToolOutput(content='4', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False), ToolCallResult(tool_name='multiply', tool_kwargs={'a': 4, 'b': 2}, tool_id='6e3a94fc-47b2-4230-928b-f5aeb8b14ac3', tool_output=ToolOutput(content='8', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 4, 'b': 2}}, raw_output=8, is_error=False), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='8', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1742761495, id='', model='Qwen/Qwen2.5-Coder-3B-Instruct', system_fingerprint='

In [60]:
ctx = Context(agent)

In [63]:
response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Your name is Bob.')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 1, 'b': 1}, tool_id='516fb245-7d47-4753-b645-0a25a1c9a948', tool_output=ToolOutput(content='2', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 1, 'b': 1}}, raw_output=2, is_error=False), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='.', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1742762526, id='', model='Qwen/Qwen2.5-Coder-3B-Instruct', system_fingerprint='3.2.1-native', usage=None, object='chat.completion.chunk'), current_agent_name='Agent')

# Creating RAG Agents with QueryEngineTools

In [67]:
# Create a vector store
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [68]:
# Create a query engine
embed_model = HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5")
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="personas",
    description="descriptions for various types of personas",
    return_direct=False,
)

/var/folders/r0/6h61qg4157lcdfpct0nslmd40000gn/T/ipykernel_8928/1451098996.py:2: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  embed_model = HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5")


In [69]:
# Create a RAG agent
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool],
    llm=llm,
    system_prompt="You are a helpful assistant that has access to a database containing persona descriptions. ",
)

In [70]:
handler = query_engine_agent.run(
    "Search the database for 'science fiction' and return some persona descriptions."
)
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: personas
Action Input: {"input": "science fiction"}
Called tool:  personas {'input': 'science fiction'} => 503, message='Service Temporarily Unavailable', url='https://router.huggingface.co/hf-inference/pipeline/feature-extraction/BAAI/bge-small-en-v1.5'
Thought: It seems there was an issue with the service. I will try again to search for 'science fiction' persona descriptions.
Action: personas
Action Input: {'input': 'science fiction'}
Called tool:  personas {'input': 'science fiction'} => 503, message='Service Temporarily Unavailable', url='https://router.huggingface.co/hf-inference/pipeline/feature-extraction/BAAI/bge-small-en-v1.5'
Thought: The service is still unavailable. I cannot retrieve the persona descriptions at this time.
Answer: It seems the service is temporarily unavailable. Please try again later.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='It seems the service is temporarily unavailable. Please try again later.')]), tool_calls=[ToolCallResult(tool_name='personas', tool_kwargs={'input': 'science fiction'}, tool_id='a0a950b3-2129-4c6e-a138-a68825228c1c', tool_output=ToolOutput(content="503, message='Service Temporarily Unavailable', url='https://router.huggingface.co/hf-inference/pipeline/feature-extraction/BAAI/bge-small-en-v1.5'", tool_name='personas', raw_input={'input': 'science fiction'}, raw_output="503, message='Service Temporarily Unavailable', url='https://router.huggingface.co/hf-inference/pipeline/feature-extraction/BAAI/bge-small-en-v1.5'", is_error=True), return_direct=False), ToolCallResult(tool_name='personas', tool_kwargs={'input': 'science fiction'}, tool_id='49eccae4-4e64-4b01-ba3c-5be9b8d907da', tool_output=ToolOutput(content="503, message='Service Temporarily Unava

# Creating multi-agent systems

In [78]:
# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

In [79]:
# Create agent configs
# NOTE: we can use FunctionAgent or ReActAgent here.
# FunctionAgent works for LLMs with a function calling API.
# ReActAgent works for any LLM.
calculator_agent = ReActAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant. Use your tools for any math operation.",
    tools=[add, subtract],
    llm=llm,
)

query_agent = ReActAgent(
    name="info_lookup",
    description="Looks up information about XYZ",
    system_prompt="Use your tool to query a RAG system to answer information about XYZ",
    tools=[query_engine_tool],
    llm=llm,
)

In [80]:
# Create and run the workflow
agent = AgentWorkflow(agents=[calculator_agent, query_agent], root_agent="calculator")

In [81]:
# Run the system
handler = agent.run(user_msg="Can you add 5 and 3?")

In [82]:
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 5, "b": 3}
Called tool:  add {'a': 5, 'b': 3} => 8
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 8

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='8')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 5, 'b': 3}, tool_id='f92d110d-ec84-4c97-b7e6-bbfc1d09bc95', tool_output=ToolOutput(content='8', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 5, 'b': 3}}, raw_output=8, is_error=False), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='8', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1742762755, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.0.1-sha-bb9095a', usage=None, object='chat.completion.chunk'), current_agent_name='calculator')